In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.gold_staging.stg_runs
USING DELTA
AS
SELECT
  time,
  run.in_num,
  run.in_name,
  run.pk
FROM workspace.silver_schema.silver_delta_table
LATERAL VIEW EXPLODE(runs) AS run

MERGE INTO workspace.gold_serving.dimension_runs AS target
USING (
  SELECT * FROM (
    SELECT
      time,
      in_num,
      in_name,
      pk,
      ROW_NUMBER() OVER (
        PARTITION BY time, in_num
        ORDER BY time DESC
      ) AS rn
    FROM workspace.gold_staging.stg_runs
  ) WHERE rn = 1
) AS source
ON target.in_num = source.in_num
AND target.time = source.time
WHEN MATCHED THEN
  UPDATE SET
    target.in_name = source.in_name,
    target.pk = source.pk
WHEN NOT MATCHED THEN
  INSERT (
    time,
    in_num,
    in_name,
    pk
  )
  VALUES (
    source.time,
    source.in_num,
    source.in_name,
    source.pk
  );

